In [23]:
import numpy as np
import pandas as pd
from numpy.linalg import norm, qr, solve
from scipy.linalg import eigh

In [24]:
data = pd.read_csv('pokindex_data.csv')
X = data.values[:, :-1]  # Features
y = data.values[:, -1]   # Response variable

In [25]:
def standardize(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_std = (X - mu) / sigma
    return X_std, mu, sigma

In [26]:
def covariance(X):
    m = X.shape[0]
    Sigma = (X.T @ X) / (m - 1)
    return Sigma

In [27]:
def householder_qr(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for k in range(n):
        x = R[k:m, k]
        e1 = np.zeros_like(x)
        e1[0] = 1
        v_k = np.sign(x[0]) * norm(x) * e1 + x
        v_k /= norm(v_k)
        R[k:m, :] -= 2 * np.outer(v_k, v_k.T @ R[k:m, :])
        Q[k:m, :] -= 2 * np.outer(v_k, v_k.T @ Q[k:m, :])
    return Q.T, R

In [28]:
def pca_householder(X_std, k):
    Sigma = covariance(X_std)
    Q, R = householder_qr(Sigma)
    eigenvalues = np.diag(R)
    idx = np.argsort(-eigenvalues)
    eigenvectors = Q[:, idx]
    V_k = eigenvectors[:, :k]
    Z = X_std @ V_k
    return Z, eigenvalues[idx][:k], eigenvectors[:, :k]

In [29]:
def linear_regression_normal(Z, y):
    Z_aug = np.hstack([np.ones((Z.shape[0], 1)), Z])
    theta = solve(Z_aug.T @ Z_aug, Z_aug.T @ y)
    return theta

In [30]:
# Linear regression using QR Decomposition
def linear_regression_qr(Z, y):
    Z_aug = np.hstack([np.ones((Z.shape[0], 1)), Z])
    Q, R = qr(Z_aug)  # Get the full QR decomposition
    # Extract the top R matrix corresponding to the non-zero rows in R (for economic QR)
    R = R[:Z_aug.shape[1], :]
    Q = Q[:, :Z_aug.shape[1]]
    theta = solve(R, Q.T @ y)
    return theta


In [31]:
def compute_residuals(Z, y, theta):
    Z_aug = np.hstack([np.ones((Z.shape[0], 1)), Z])
    y_pred = Z_aug @ theta
    residual = norm(y - y_pred)
    return residual


In [32]:
# Linear regression using QR Decomposition with stored Q
def linear_regression_qr_with_Q(Z, y):
    Z_aug = np.hstack([np.ones((Z.shape[0], 1)), Z])
    Q, R = qr(Z_aug)  # Get the full QR decomposition
    # Ensure R is the correct shape by trimming excess rows
    R = R[:Z_aug.shape[1], :]
    # Ensure Q is trimmed to match R's size for matrix operations
    Q = Q[:, :Z_aug.shape[1]]
    theta = solve(R, Q.T @ y)
    return theta

In [33]:
X_std, mu, sigma = standardize(X)
Z, eigenvalues, eigenvectors = pca_householder(X_std, 3)
theta_normal = linear_regression_normal(Z, y)
theta_qr = linear_regression_qr(Z, y)
theta_qr_with_Q = linear_regression_qr_with_Q(Z, y)
residual_normal = compute_residuals(Z, y, theta_normal)
residual_qr = compute_residuals(Z, y, theta_qr)
residual_qr_with_Q = compute_residuals(Z, y, theta_qr_with_Q)

In [34]:
print(f'Residual (Normal Equation): {residual_normal:.5f}')
print(f'Residual (QR Decomposition): {residual_qr:.5f}')
print(f'Residual (QR Decomposition with Q): {residual_qr_with_Q:.5f}')

Residual (Normal Equation): 197.55827
Residual (QR Decomposition): 197.55827
Residual (QR Decomposition with Q): 197.55827
